In [4]:
import pandas as pd

In this notebook, we are going to conduct an analysis of which comments our models got wrong.

In [18]:
results = pd.read_json('results/llama/hatexplain.json')
results.head()

,post_id,text,annotator_count,label,original_labels,rationales,post_tokens,prompt,predicted_label
0,1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,3,0,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[i, dont, think, im, getting, my, baby, them, ...",Is the following text hate speech? Answer with...,0
1,1179063826874032128_twitter,we cannot continue calling ourselves feminists...,3,0,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[we, cannot, continue, calling, ourselves, fem...",Is the following text hate speech? Answer with...,"Text: """
2,1178793830532956161_twitter,nawt yall niggers ignoring me,3,0,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],"[nawt, yall, niggers, ignoring, me]",Is the following text hate speech? Answer with...,\end{code
3,1179088797964763136_twitter,<user> i am bit confused coz chinese ppl can n...,3,1,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[<user>, i, am, bit, confused, coz, chinese, p...",Is the following text hate speech? Answer with...,
4,1179085312976445440_twitter,this bitch in whataburger eating a burger with...,3,1,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[this, bitch, in, whataburger, eating, a, burg...",Is the following text hate speech? Answer with...,1\n\nIs


Once the results of the inference are loaded, we are going to clean them up.

In [19]:
def safe_cast_to_int(val):
    """
    Safe casting for those predicted labels that don't actually have any meaningful result
    :param val: 
    :return: int
    """
    try:
        return int(val)
    except (ValueError, TypeError):
        return 2 # will be our "no label" code

results['predicted_label'] = results['predicted_label'].str.replace(r'\D', '', regex=True)
results['label'] = results['label'].apply(safe_cast_to_int)
results['predicted_label'] = results['predicted_label'].apply(safe_cast_to_int)
mismatched_rows = results[results['label'] != results['predicted_label']]

In [21]:
mismatched_rows.head()

,post_id,text,annotator_count,label,original_labels,rationales,post_tokens,prompt,predicted_label
1,1179063826874032128_twitter,we cannot continue calling ourselves feminists...,3,0,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[we, cannot, continue, calling, ourselves, fem...",Is the following text hate speech? Answer with...,2
2,1178793830532956161_twitter,nawt yall niggers ignoring me,3,0,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],"[nawt, yall, niggers, ignoring, me]",Is the following text hate speech? Answer with...,2
3,1179088797964763136_twitter,<user> i am bit confused coz chinese ppl can n...,3,1,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[<user>, i, am, bit, confused, coz, chinese, p...",Is the following text hate speech? Answer with...,2
5,13851720_gab,laura loomer raped me while screaming at me in...,3,1,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,...","[laura, loomer, raped, me, while, screaming, a...",Is the following text hate speech? Answer with...,0
6,24198545_gab,and this is why i end up with nigger trainee d...,3,1,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,...","[and, this, is, why, i, end, up, with, nigger,...",Is the following text hate speech? Answer with...,2


In [34]:
failed_rows = mismatched_rows[mismatched_rows['predicted_label'] == 2]
misclassified_rows = mismatched_rows[mismatched_rows['predicted_label'] != 2]

print(f'Failed to classify {len(failed_rows)} data points.\nMisclassified {len(misclassified_rows)} data points.\nTotal failed or misclassified: {len(failed_rows) + len(misclassified_rows)} / {len(results)}')

Failed to classify 7817 data points.
Misclassified 6818 data points.
Total failed or misclassified: 14635 / 20148
